In [1]:
# import numpy as np
import pandas as pd
import requests
import time
import datetime

import env

In [7]:
search_base_url = 'https://www.googleapis.com/youtube/v3/search'
videos_base_url = 'https://www.googleapis.com/youtube/v3/videos'

search_params = {
  'key': env.token,
  'part': 'snippet',
  'order': 'viewCount',
  'regionCode': 'KR',
  'type': 'video',
  'maxResults': 20,
  'fields': 'items(id/videoId, snippet(publishedAt,title))'
}
videos_params = {
  'key': env.token,
  'part': 'statistics',
  'fields': 'items/statistics'
}

search_keywords = ['영화', '명장면', '리뷰', '해석', '의미']
columns = ['영상 id', '영화 제목', '검색 키워드', '영상 제목', '조회수', '좋아요수', '댓글 수', '생성된 날짜']

tokenLimit = len(env.token)-1

In [8]:
movieInfo = pd.read_csv('movieInfo.csv').loc[:,['영화명', '개봉일']]
movieNames = list(movieInfo.loc[:, '영화명'])

In [9]:

def get_video_list():
  columns = ['영상 id', '영화 제목', '검색 키워드', '영상 제목', '조회수', '좋아요수', '댓글 수', '생성된 날짜']
  video_list = pd.DataFrame(columns=columns)
  tokenNum = 0

  for movieName in movieNames[12:]:
    print(movieName, len(video_list))
    for keyword in search_keywords:
      # parameter 설정
      publishedAfter = list(movieInfo[movieInfo['영화명'] == movieName].loc[:, '개봉일'])[0]
      year, month, day = publishedAfter.split('-')
      publishedBefore = (datetime.datetime(int(year), int(month), int(day)) + datetime.timedelta(days=30)).strftime('%Y-%m-%d')
      q = ' '.join([movieName, keyword])

      search_params['q'] = q
      search_params['publishedAfter'] = publishedAfter+'T00:00:00Z'
      search_params['publishedBefore'] = publishedBefore+'T00:00:00Z'

      time.sleep(10)
      res_search = requests.get(search_base_url, search_params).json()

      # 네트워크 오류 발생 시
      while 'error' in res_search.keys():
        print('[ERROR] search requests ', res_search)
        tokenNum += 1
        if tokenNum > tokenLimit: 
          print('[Error] token 오늘 할당량 끝났다~~', movieName, keyword)
          return video_list

        print('[HTTPError] token 교체')
        search_params['key'] = env.token[tokenNum]
        res_search = requests.get(search_base_url, search_params).json()

      # 응답 데이터 가공
      if 'items' in res_search.keys():
        for item in res_search['items']:
          result_video_ids = list(video_list.loc[video_list['영화 제목'] == movieName,'영상 id'])

          if item['id']['videoId'] not in result_video_ids:
            videos_params['id'] = item['id']['videoId']
            res_videos = requests.get(videos_base_url, videos_params).json()

            # 네트워크 오류 발생 시
            while 'error' in res_videos.keys():
              print('[ERROR] video requests ', res_videos)
              tokenNum += 1
              if tokenNum > tokenLimit: 
                print('[Error] token 오늘 할당량 끝났다~~', movieName)
                return video_list

              print('[HTTPError] token 교체')
              search_params['key'] = env.token[tokenNum]
              res_videos = requests.get(videos_base_url, videos_params).json()

            if 'items' in res_videos.keys():
              try:
                video_statistics = res_videos['items'][0]['statistics']
                item_data = [[
                  item['id']['videoId'], 
                  movieName, 
                  q, 
                  item['snippet']['title'], 
                  int(video_statistics['viewCount']) if 'viewCount' in video_statistics else 0,
                  int(video_statistics['likeCount']) if 'likeCount' in video_statistics else 0,
                  int(video_statistics['commentCount']) if 'commentCount' in video_statistics else 0,
                  item['snippet']['publishedAt'], 
                ]]
                video_list = video_list.append(pd.DataFrame(item_data, columns=columns),ignore_index=True)
              except Exception as e:
                print('[ERROR] ', e, movieName, res_videos)
                return video_list


  return video_list

In [10]:
video_list = get_video_list()

돈 0
악인전 38
가장 보통의 연애 74
말모이 109
증인 145
블랙머니 179
터미네이터: 다크 페이트 213
사바하 244
타짜: 원 아이드 잭 281
신의 한 수: 귀수편 308
알리타: 배틀 엔젤 346
내안의 그놈 384
뺑반 409
변신 443
주먹왕 랄프 2: 인터넷 속으로 474
[ERROR] video requests  {'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]}}
[HTTPError] token 교체
[ERROR] video requests  {'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]}}
[HTTPError] token 교체
[ERROR] video requests  {'er

In [36]:
result = video_list.loc[video_list['영화 제목'] != '주먹왕 랄프 2: 인터넷 속으로',]
result = result.set_index('영상 id')

In [25]:
movieNames.index('주먹왕 랄프 2: 인터넷 속으로')

26

In [38]:
result.to_csv('./videoList/movieVideoList_12_25.csv', encoding='utf-8-sig')

In [39]:
top = lambda x: x.sort_values(by='조회수', ascending=False)[:10]
filtered_result = result.groupby('영화 제목').apply(top)
filtered_result

영화 제목            검색 키워드  \
영화 제목         영상 id                                          
가장 보통의 연애     A6Vn67Sdmig      가장 보통의 연애      가장 보통의 연애 영화   
              QPAHDHyxyiQ      가장 보통의 연애      가장 보통의 연애 영화   
              wwHeVWY_osk      가장 보통의 연애      가장 보통의 연애 영화   
              vBGnr2i_EeI      가장 보통의 연애      가장 보통의 연애 영화   
              PXzXRQYkCQs      가장 보통의 연애      가장 보통의 연애 영화   
...                                  ...               ...   
터미네이터: 다크 페이트 j_13slAOO6k  터미네이터: 다크 페이트  터미네이터: 다크 페이트 영화   
              -kNzfvM4dGA  터미네이터: 다크 페이트  터미네이터: 다크 페이트 리뷰   
              QFBbDjJIleU  터미네이터: 다크 페이트  터미네이터: 다크 페이트 리뷰   
              61Y6G8M6QsQ  터미네이터: 다크 페이트  터미네이터: 다크 페이트 영화   
              8KjnAvJl0Pk  터미네이터: 다크 페이트  터미네이터: 다크 페이트 리뷰   

                                                                       영상 제목  \
영화 제목         영상 id                                                            
가장 보통의 연애     A6Vn67Sdmig            사랑은 진동하는 소주 냄새를 타고 이뤄지는 것: 가장 보통의 연애 리뷰   
              QPAHDHyxyiQ  서로 안 맞는 공효진, 김래원, 당황 박지선 : Gong Hyo-jin, Kim R...   
              wwHeVWY_osk                           [가장 보통의 연애] 전국민 격공 리뷰 영상   
              vBGnr2i_EeI                                  영화 [가장 보통의 연애] 리뷰   
              PXzXRQYkCQs  공효진 웃겨서 힘들게(?) 한 강기영의 애드립 장면 : Gong Hyo-jin, K...   
...                                                                      ...   
터미네이터: 다크 페이트 j_13slAOO6k          터미네이터 다크 페이트 리뷰 (감독 친필 싸인 피규어 이벤트, 스포 포함)   
              -kNzfvM4dGA  【브롤스타즈】 스토리리뷰 102화 &quot;터미네이터 : 다크페이트 #2&quot...   
              QFBbDjJIleU  【브롤스타즈】 스토리리뷰 101화 &quot;터미네이터 : 다크페이트 #1&quot...   
              61Y6G8M6QsQ                          자신의 얼굴에 똥칠하고도 폭망한 제임스 카메론   
              8KjnAvJl0Pk     (시리즈요약) 인간을 노예로삼은 미래의 기계들ㅣ터미네이터 1부ㅣ(영화리뷰,결말포함)   

                              조회수  좋아요수  댓글 수                생성된 날짜  
영화 제목         영상 id                                                  
가장 보통의 연애     A6Vn67Sdmig   88459  1098   221  2019-10-10T10:00:02Z  
              QPAHDHyxyiQ   67447   404    25  2019-10-02T05:20:07Z  
              wwHeVWY_osk   65206    35     0  2019-10-02T10:04:13Z  
              vBGnr2i_EeI   62709   261    64  2019-10-02T07:30:03Z  
              PXzXRQYkCQs   27930   136     7  2019-10-02T06:16:43Z  
...                           ...   ...   ...                   ...  
터미네이터: 다크 페이트 j_13slAOO6k  176513  2731  1230  2019-10-30T09:27:08Z  
              -kNzfvM4dGA  153875  1708   445  2019-11-02T06:00:02Z  
              QFBbDjJIleU  134702  1640   461  2019-11-01T06:00:00Z  
              61Y6G8M6QsQ  131103  2610   823  2019-11-25T10:00:01Z  
              8KjnAvJl0Pk  124537   802   170  2019-11-22T09:25:40Z  

[140 rows x 7 columns]

In [40]:
filtered_result.to_csv('./filteredVideoList/movieVideoList_12_25.csv', encoding='utf-8-sig')